## WCS solutions

### Exercise: Refine a WCS using a list of detections and a reference catalog

Refine a WCS for a science image exposure from the Zwicky Transient Facility from these ingredients:
* An initial header
* A detection list cut at 17th magnitude, in file `data/ztf_detections_17thmag.csv`
* A reference catalog with coordinates and magnitudes from Gaia cut at 17 Gaia G magnitude, in `data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv`

The exercise makes use of `astropy.wcs`, `astropy.coordinates` and the projection capabilities of WCSAxes.

1. Read in the detection list and the reference catalog with `astropy.table.Table.read`
2. Calculate starting RAs and Decs for the detection list using the initial WCS
3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates
4. Plot the detection list and the Gaia list in a scatter plot
5. Match the detection list and the Gaia list
6. Refine the WCS using the `fit_wcs_from_points` function from `astropy.wcs.utils`

Import everything we'll need for the exercise.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
from astropy.wcs.utils import fit_wcs_from_points
import astropy.units as u

%matplotlib inline

Create the initial WCS programatically.

In [ ]:
initial_wcs = WCS(naxis=2)
initial_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']  
initial_wcs.wcs.crval = [149.07662386535503, 33.32164150821777]  
initial_wcs.wcs.crpix = [-3305.678, -7136.481]
initial_wcs.wcs.cd = [[-0.0002817188, -1.554e-07],
                      [-1.998e-07, -0.0002819204]]  
initial_wcs.array_shape = [3080, 3072] # NAXIS2, NAXIS1

In [ ]:
initial_wcs

### 1. Read in the detection list and the reference catalog

Read in the detections and the reference catalog using `astropy.table.Table` with `format='csv'`.
The detections table is in `'data/ztf_detections_17thmag.csv'` and the reference catalog is `'data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv'`

In [ ]:
detections = Table.read(os.path.join('data', 'ztf_detections_17thmag.csv'), format='csv')

In [ ]:
ref_catalog = Table.read(os.path.join('data', 'Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv'))

### 2. Calculate starting RAs and Decs for the detection list using the initial WCS

Use the `initial_wcs.all_pix2world` function to calculate starting RA and Dec from the `detections['xpos']` and `detections['ypos']` columns. The pixel positions use the FITS numbering convention.

In [ ]:
initial_ra, initial_dec = initial_wcs.all_pix2world(detections['xpos'], detections['ypos'], 1)

### 3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates

In [ ]:
initial_coords = SkyCoord(ra=initial_ra, dec=initial_dec, unit=u.deg)

In [ ]:
gaia_coords = SkyCoord(ra=ref_catalog['ra'], dec=ref_catalog['dec'], unit=u.deg)

### 4. Plot the detection list and the Gaia list in a scatter plot

Use `projection=initial_wcs` to make a scatter plot using `gaia_coords` and `initial_coords`. The open circles are sized according to magnitude.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=initial_wcs)

ax.scatter(gaia_coords.ra, 
           gaia_coords.dec,  c=None, marker='o',
           s=20*(18 - ref_catalog['phot_g_mean_mag']),
           facecolors='none', edgecolors='green',
           transform=ax.get_transform('world'))

ax.scatter(initial_coords.ra, 
           initial_coords.dec, c=None, marker='o',
           s=20*(18 - detections['mag']),
           facecolors='none', edgecolors='blue',
           transform=ax.get_transform('world'))



### 5. Match the detection list and the Gaia list

Use the `initial_coords.search_around_sky` method with a 15 arcsecond radius.

In [ ]:
idxgaia, idxdet, d2d, d3d = initial_coords.search_around_sky(gaia_coords, 15*u.arcsec)

In [ ]:
gaia_matched = gaia_coords[idxgaia]
detections_xpos_matched = detections['xpos'][idxdet]
detections_ypos_matched = detections['ypos'][idxdet]
print(len(gaia_matched), len(detections_xpos_matched), len(detections_ypos_matched))

### 6. Refine the WCS using the `fit_wcs_from_points` function

Look at the help for `fit_wcs_from_points` and use it to fit a new WCS.

Optionally, calculate new RAs and Decs for the matched pixel coordinates, and make another scatter plot.

In [ ]:
fit_wcs_from_points?

In [ ]:
fitted_wcs = fit_wcs_from_points((detections_xpos_matched, detections_ypos_matched),
                                 gaia_matched,
                                 projection='TAN', sip_degree=3)

In [ ]:
fitted_wcs

Examine the SIP distortion coefficients

In [ ]:
fitted_wcs.sip.a

In [ ]:
fitted_wcs.sip.b

Optionally, calculate new RAs and Decs for the matched pixel coordinates, and make another scatter plot.

In [ ]:
fitted_ra, fitted_dec = fitted_wcs.all_pix2world(detections_xpos_matched,
                                                 detections_ypos_matched, 1)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=fitted_wcs)

ax.scatter(gaia_matched.ra, 
           gaia_matched.dec,  c=None, marker='o',
           s=20*(18 - ref_catalog['phot_g_mean_mag'][idxgaia]),
           facecolors='none', edgecolors='green',
           transform=ax.get_transform('world'))

ax.scatter(fitted_ra, 
           fitted_dec,  c=None, marker='o',
           s=20*(18 - detections['mag'][idxdet]),
           facecolors='none', edgecolors='blue',
           transform=ax.get_transform('world'))

